# Semantisk Kerne

I dette kodeeksempel vil du bruge [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI-rammeværket til at oprette en grundlæggende agent.

Målet med dette eksempel er at vise dig de trin, som vi senere vil bruge i de yderligere kodeeksempler, når vi implementerer de forskellige agentmønstre.


## Importer de nødvendige Python-pakker


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Oprettelse af klienten

I dette eksempel vil vi bruge [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for adgang til LLM.

`ai_model_id` er defineret som `gpt-4o-mini`. Prøv at ændre modellen til en anden model, der er tilgængelig på GitHub Models-markedspladsen, for at se de forskellige resultater.

For at bruge `Azure Inference SDK`, som anvendes til `base_url` for GitHub Models, vil vi bruge `OpenAIChatCompletion`-connectoren inden for Semantic Kernel. Der findes også andre [tilgængelige connectorer](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), som kan bruges med Semantic Kernel til andre modeludbydere.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Oprettelse af Agenten

Her opretter vi agenten kaldet `TravelAgent`.

I dette eksempel bruger vi meget enkle instruktioner. Du kan ændre disse instruktioner for at se, hvordan agenten reagerer anderledes.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Kørsel af agenten

Nu kan vi køre agenten ved at definere en tråd af typen `ChatHistoryAgentThread`. Eventuelle nødvendige systemmeddelelser gives som argumentet `messages` til agentens invoke_stream.

Når disse er defineret, opretter vi en `user_inputs`, som repræsenterer det, brugeren sender til agenten. I dette tilfælde har vi sat denne besked til `Planlæg en solrig ferie`.

Du er velkommen til at ændre denne besked for at se, hvordan agenten reagerer anderledes.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at sikre nøjagtighed, skal det bemærkes, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
